In [3]:
import os
import yaml
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from composio_langchain import ComposioToolSet, App
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool
from langchain_core.tools import ToolException
from typing import List, Optional, Dict, Any
import warnings

warnings.filterwarnings("ignore")

In [4]:
class MyAgent:
    def __init__(self, tools, custom_prompt: str = None):
        self.llm = ChatOpenAI(
            model="gpt-4o",
            api_key=os.getenv("OPENAI_API_KEY"),
        )
        self.tools = tools
        self.agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent = create_tool_calling_agent(
            llm=self.llm,
            tools=self.enhace_agent_tools(tools),
            prompt=ChatPromptTemplate.from_messages([
                ("system", custom_prompt if custom_prompt else "You are an experienced assistant. You are able to perform any task asked by the user through all the tools you have access to. Your goal is to complete the user's task by using the tools you have access to."),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]),
        )
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, memory=self.agent_memory, verbose=True)

    def myinvoke(self, input: str):
        result = self.executor.invoke({"input": input})
        print(result)
        print("\n==================================================")
        print("--> Agent Response:")
        print(result['output'])
        print("\n==================================================")

    def try_except_tool(self, func):
        try:
            return func
        except ToolException as e:
            print(f"Error: {e}")

    def enhace_agent_tools(self, agent_tools: List):
        enhanced_agent_tools = [
            StructuredTool(
                name=tool.name,
                description=tool.description,
                args_schema=tool.args_schema,
                func=self.try_except_tool(tool.func),
                handle_tool_error=True,
                handle_validation_error=True
            ) for tool in agent_tools
        ]
        return enhanced_agent_tools

In [5]:
composio_toolset = ComposioToolSet(api_key=os.getenv('COMPOSIO_API_KEY'))
agent_tools = composio_toolset.get_tools(apps=[App.CLICKUP])

In [6]:
agent_tools

[StructuredTool(name='clickup_attachments_upload_file_to_task_as_attachment', description='Upload a file to a task as an attachment.  ***Note:** This request uses multipart/form-data as the content type.*', args_schema=<class 'pydantic.v1.main.attachments_upload_file_to_task_as_attachmentRequest'>, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x12292eb60>),
 StructuredTool(name='clickup_authorization_get_access_token', description="These are the routes for authing the API and going through the [OAuth flow](https://clickup.com/api). Applications utilizing a personal API token don't use this endpoint. ***Note:** OAuth tokens are not supported when using the [**Try It** feature](https://clickup.com/api) of our Reference docs. You can't try this endpoint from your web browser.*", args_schema=<class 'pydantic.v1.main.authorization_get_access_tokenRequest'>, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x12292fc40>),
 StructuredTool(name='clickup_autho

In [9]:
agent = MyAgent(tools=agent_tools)

In [10]:
agent.myinvoke("Retrieve the name of my clickup workspace")



> Entering new AgentExecutor chain...

Invoking: `clickup_authorization_view_account_details` with `{}`


{'execution_details': {'executed': True}, 'response_data': {'user': {'id': 81918955, 'username': 'João Guilherme Silva Morossini', 'email': 'joaog.morossini@gmail.com', 'color': '#81b1ff', 'profilePicture': 'https://attachments.clickup.com/profilePictures/81918955_kcK.jpg', 'initials': 'JM', 'week_start_day': 1, 'global_font_support': True, 'timezone': 'America/Sao_Paulo'}}}
Invoking: `clickup_authorization_get_work_space_list` with `{}`


{'execution_details': {'executed': True}, 'response_data': {'teams': [{'id': '12927880', 'name': 'Cogmo Workspace', 'color': '#202020', 'avatar': 'https://attachments2.clickup.com/team_avatars/12927880_9xT.blob?Expires=1735603200&Key-Pair-Id=APKAIYJRUKB5RLQWHZWQ&Signature=IjQQWt2qYTYv5yXQn3jhIfm5i9-ETIQdodisZqlZDGzoY4qAE5qvv~0FHDxbzSqeN0Rd-osZw5sJIpIpYWO1SPzvm2u-gVJgy9RGaaB4KtnMyZw-wStnG67hevdPdOukxHFE~VLEEP2HoHSnYyWyjUrMFmA0f256ZsU8hhM4Lq8JuUf

In [11]:
clickup_team_id="12927880"

In [12]:
agent.myinvoke(f"Which tools do you have for working with Clickup spaces?")



> Entering new AgentExecutor chain...
I have several tools for working with ClickUp spaces. Here are the available tools:

1. **View the Spaces available in a Workspace**:
   - `clickup_spaces_get_space_details`: View the Spaces available in a Workspace.

2. **Add a new Space to a Workspace**:
   - `clickup_spaces_add_new_space_to_work_space`: Add a new Space to a Workspace.

3. **Update Space Details and Enable ClickApps**:
   - `clickup_spaces_update_details_and_enable_click_apps`: Rename, set the Space color, and enable ClickApps for a Space.

4. **Delete a Space from your Workspace**:
   - `clickup_spaces_remove_space`: Delete a Space from your Workspace.

5. **View the Spaces available in a Workspace** (Alternative method):
   - `clickup_spaces_get_details`: View the Spaces available in a Workspace.

If you need to perform any specific action with ClickUp spaces, please let me know!

> Finished chain.
{'input': 'Which tools do you have for working with Clickup spaces?', 'chat_hi

In [13]:
agent.myinvoke(f"Use 'clickup_spaces_get_space_details' with the following team_id to view available Spaces in my Workspace. team_id: {clickup_team_id}")



> Entering new AgentExecutor chain...

Invoking: `clickup_spaces_get_space_details` with `{'team_id': 12927880}`


{'execution_details': {'executed': False}, 'response_data': {'error': '{"err":"invalid input syntax for integer: \\"12927880.0\\"","ECODE":"SHARD_001"}', 'status_code': 500}}It seems there was an error with the input syntax for the `team_id` provided. It might be due to an extra character or incorrect format. Kindly verify the `team_id` and provide it again, or let me know if there's anything else I can assist you with.

> Finished chain.
{'input': "Use 'clickup_spaces_get_space_details' with the following team_id to view available Spaces in my Workspace. team_id: 12927880", 'chat_history': [HumanMessage(content='Retrieve the name of my clickup workspace'), AIMessage(content='The name of your ClickUp workspace is **"Cogmo Workspace"**.'), HumanMessage(content='Which tools do you have for working with Clickup spaces?'), AIMessage(content='I have several tools for working 

In [14]:
agent.myinvoke("Use 'clickup_spaces_get_space_details' with the following team_id to view available Spaces in my Workspace. team_id:'12927880'. You MUST pass the team_id as an integer, NOT as a float or as a string")



> Entering new AgentExecutor chain...

Invoking: `clickup_spaces_get_space_details` with `{'team_id': 12927880}`


{'execution_details': {'executed': False}, 'response_data': {'error': '{"err":"invalid input syntax for integer: \\"12927880.0\\"","ECODE":"SHARD_001"}', 'status_code': 500}}
Invoking: `clickup_spaces_get_space_details` with `{'team_id': 12927880}`
responded: It seems there was an issue with the request. The error message indicates an invalid input syntax for the integer.

Let's try again with the team ID formatted correctly as an integer.

{'execution_details': {'executed': False}, 'response_data': {'error': '{"err":"invalid input syntax for integer: \\"12927880.0\\"","ECODE":"SHARD_001"}', 'status_code': 500}}
Invoking: `clickup_spaces_get_details` with `{'space_id': 12927880}`
responded: It appears that the API is returning an error related to the input syntax for the integer, even though the `team_id` was correctly passed as an integer.

There might be an issue on th

In [15]:
agent.myinvoke("What can you tell me about my Clickup space of 54804921?")



> Entering new AgentExecutor chain...

Invoking: `clickup_spaces_get_details` with `{'space_id': 54804921}`


{'execution_details': {'executed': False}, 'response_data': {'error': '{"err":"Space ID invalid","ECODE":"INPUT_002"}', 'status_code': 400}}It appears that the Space ID `54804921` is invalid. Please verify the Space ID and try again, or let me know if there's any other information or action you would like to pursue.

> Finished chain.
{'input': 'What can you tell me about my Clickup space of 54804921?', 'chat_history': [HumanMessage(content='Retrieve the name of my clickup workspace'), AIMessage(content='The name of your ClickUp workspace is **"Cogmo Workspace"**.'), HumanMessage(content='Which tools do you have for working with Clickup spaces?'), AIMessage(content='I have several tools for working with ClickUp spaces. Here are the available tools:\n\n1. **View the Spaces available in a Workspace**:\n   - `clickup_spaces_get_space_details`: View the Spaces available in a Work

In [16]:
agent.myinvoke("Can you give me a list of Spaces in my clickup workspace?")



> Entering new AgentExecutor chain...

Invoking: `clickup_authorization_get_work_space_list` with `{}`


{'execution_details': {'executed': True}, 'response_data': {'teams': [{'id': '12927880', 'name': 'Cogmo Workspace', 'color': '#202020', 'avatar': 'https://attachments2.clickup.com/team_avatars/12927880_9xT.blob?Expires=1735603200&Key-Pair-Id=APKAIYJRUKB5RLQWHZWQ&Signature=IjQQWt2qYTYv5yXQn3jhIfm5i9-ETIQdodisZqlZDGzoY4qAE5qvv~0FHDxbzSqeN0Rd-osZw5sJIpIpYWO1SPzvm2u-gVJgy9RGaaB4KtnMyZw-wStnG67hevdPdOukxHFE~VLEEP2HoHSnYyWyjUrMFmA0f256ZsU8hhM4Lq8JuUfFJyi69HbCD2l~F8kCeLOadW4z0ofkTTMfVNjDrqezFkiL72Rlr3v~bmq3oCGKIaAMZFWyorq3ru5HDJvphqudIkrFpA56kIgESPIzzU3lkINk7CVHJ2xKRRnU4ruo4cjufxo5RAtL9RehP5gaP8r2A1BmgDlZ5uLTUiK0kg__', 'members': [{'user': {'id': 44309342, 'username': 'Hibrael Loures', 'email': 'ibraelmoura@gmail.com', 'color': None, 'profilePicture': 'https://attachments.clickup.com/profilePictures/44309342_YHC.jpg', 'initials': 'HL', 'role': 3, 'custom_role': None, 'last_active': '1719

In [17]:
agent.myinvoke("Retrieve a list of members in my workspace")



> Entering new AgentExecutor chain...
Unfortunately, I don't have direct access to a specific tool for retrieving a list of members in your ClickUp workspace. However, I can help you with tasks related to managing users, such as inviting users, updating user details, or deactivating users in your workspace.

If you have any specific tasks or actions you'd like to perform regarding workspace members, please let me know! Alternatively, you can use the ClickUp web interface to view and manage members in your workspace.

> Finished chain.
{'input': 'Retrieve a list of members in my workspace', 'chat_history': [HumanMessage(content='Retrieve the name of my clickup workspace'), AIMessage(content='The name of your ClickUp workspace is **"Cogmo Workspace"**.'), HumanMessage(content='Which tools do you have for working with Clickup spaces?'), AIMessage(content='I have several tools for working with ClickUp spaces. Here are the available tools:\n\n1. **View the Spaces available in a Workspace*

In [41]:
agent_tools[50]

StructuredTool(name='clickup_lists_create_folder_less_list', description='Add a new List in a Space.', args_schema=<class 'pydantic.v1.main.lists_create_folder_less_listRequest'>, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x122c323e0>)

In [19]:
agent_tools[0].func

<function composio_langchain.toolset.ComposioToolSet._wrap_action.<locals>.function(task_id: str, custom_task_ids: bool = None, team_id: float = None)>

In [23]:
type(agent_tools)

list

In [46]:
for i, tool in enumerate(agent_tools):
    if "clickup_lists_get_list_details" in tool.name:
        pprint(f"{i}: {tool}")

("51: name='clickup_lists_get_list_details' description='View information "
 "about a List.' args_schema=<class "
 "'pydantic.v1.main.lists_get_list_detailsRequest'> func=<function "
 'ComposioToolSet._wrap_action.<locals>.function at 0x122c32980>')


In [51]:
import inspect
from pprint import pprint

# Retrieve the Pydantic model class for the tool at index 51
model_class = agent_tools[51].args_schema
try:
    # Attempt to print the source code of the Pydantic model class
    source_code = inspect.getsource(model_class)
    pprint(source_code)
except OSError:
    # Handle the case where the source code cannot be found
    pprint(f"Could not find the source code for the class {model_class.__name__}")

'Could not find the source code for the class lists_get_list_detailsRequest'


In [43]:
agent_tools[51].func

<function composio_langchain.toolset.ComposioToolSet._wrap_action.<locals>.function(list_id: float)>

In [29]:
import inspect
from pprint import pprint

function = inspect.getsource(agent_tools[60].func)
pprint(function)

('        def function(**kwargs: t.Any) -> t.Dict:\n'
 '            """Wrapper function for composio action."""\n'
 '            return self.execute_action(\n'
 '                action=Action(value=action),\n'
 '                params=kwargs,\n'
 '                entity_id=entity_id or self.entity_id,\n'
 '            )\n')


In [26]:
from composio_langchain.toolset import ComposioToolSet

composio_toolset = ComposioToolSet(api_key=os.getenv('COMPOSIO_API_KEY'))
composio_toolset

In [27]:
dir(composio_toolset)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_connected_accounts',
 '_execute_local',
 '_execute_remote',
 '_local_client',
 '_logger',
 '_remote_client',
 '_runtime',
 '_save_var_files',
 '_serialize_execute_params',
 '_try_get_github_access_token_for_current_entity',
 '_workspace',
 '_workspace_config',
 '_workspace_id',
 '_wrap_action',
 '_wrap_tool',
 '_write_to_file',
 'action_preprocessing',
 'api_key',
 'base_url',
 'check_connected_account',
 'client',
 'create_trigger_listener',
 'entity_id',
 'execute_action',
 'find_actions_by_tags',
 'find_actions_by_use_case',
 'get_action_schemas',
 'get_actions',
 'get_agent_instr

In [52]:
composio_toolset.get_action_schemas()

[]